<a href="https://colab.research.google.com/github/NaseemaO/Deep_Learning_Challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.2 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf
%matplotlib inline
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import files
import matplotlib.pyplot as plt

In [3]:
#  Import, read the charity_data.csv and save to a DataFrame 'application_df'.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   EIN                     34299 non-null  int64 
 1   NAME                    34299 non-null  object
 2   APPLICATION_TYPE        34299 non-null  object
 3   AFFILIATION             34299 non-null  object
 4   CLASSIFICATION          34299 non-null  object
 5   USE_CASE                34299 non-null  object
 6   ORGANIZATION            34299 non-null  object
 7   STATUS                  34299 non-null  int64 
 8   INCOME_AMT              34299 non-null  object
 9   SPECIAL_CONSIDERATIONS  34299 non-null  object
 10  ASK_AMT                 34299 non-null  int64 
 11  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 3.1+ MB


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME' the identifiers of the Organization. Each Row is an Organization after the initial column header row.
application_df.drop(application_df.columns[[0, 1]], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning. # create a list to hold the application_type
app_type_bin = application_df["APPLICATION_TYPE"].value_counts()
app_type_bin

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_cutoff_value =527

# Create a list of application types to be replaced
application_types_to_replace = app_type_bin[app_type_bin < app_cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
class_bin = application_df["CLASSIFICATION"].value_counts()
class_bin

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_bin[class_bin> 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_cutoff_value = 1882

classifications_to_replace = class_bin[class_bin < class_cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [12]:
# identify the dtypes in application_df
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [13]:
## Generate our categorical variable lists that are with the dtype Object.
app_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
app_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [14]:
# Check the number of unique values in each column.  After binning, none of the Features have >= 10 unique values.
application_df[app_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=application_df.dtypes[application_df.dtypes == "object"].index.tolist())

# Display the modified DataFrame.
display(application_df.head())
display(application_df.tail())

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
34298,1,36500179,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0


In [17]:
# Split our preprocessed data into our features and target arrays.  X is the features. y is the target.
X = application_df.drop(columns = 'IS_SUCCESSFUL').values
y = application_df.IS_SUCCESSFUL.values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [18]:
# Display the shapes of the resulting datasets.
# there are 43 columns / Features
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (25724, 43)
X_test shape: (8575, 43)
y_train shape: (25724,)
y_test shape: (8575,)


In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# This is for the nn_model the Model 1 done in the first ipynb file. Commented out.  Define the model - deep neural network, i.e., the number of input features and hidden nodes for each layer.
# # 80 neurons in first hidden layer1, 30 in second hidden layer2 based on output in starte#r code for this code block

# input_features = len(X_train[0])

# hidden_nodes_layer1 =  80
# hidden_nodes_layer2 = 30

# nn_model = tf.keras.models.Sequential()

# # First hidden layer
# nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# # Second hidden layer
# nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# # Output layer
# nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn_model.summary()

In [ ]:
# # Compile the model
# nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
# fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data
# model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Model 1 Loss: {model_loss}, Model 1 Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5602 - accuracy: 0.7256 - 396ms/epoch - 1ms/step
Model 1 Loss: 0.5601772665977478, Model 1 Accuracy: 0.7255976796150208


In [ ]:
# # Save the entire model as a `.keras`  or zip archive.
# nn_model.save('AlphabetSoupCharity_model.keras')
# files.download('AlphabetSoupCharity_model.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Optimization Model 2. with Hyperparameter Options to get the best model hyperparameters, and identify the top 3 Optimization models.

In [20]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):

    input_features = len(X_train[0])

    model_2 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    model_2.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=5), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        model_2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    model_2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return model_2

In [21]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [22]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 02m 24s]
val_accuracy: 0.7229154706001282

Best val_accuracy So Far: 0.7268804907798767
Total elapsed time: 00h 36m 54s


In [23]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 6, 'num_layers': 5, 'units_0': 1, 'units_1': 1, 'units_2': 9, 'units_3': 9, 'units_4': 9, 'units_5': 3, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 6, 'num_layers': 5, 'units_0': 1, 'units_1': 1, 'units_2': 9, 'units_3': 9, 'units_4': 9, 'units_5': 3, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0022'}
{'activation': 'sigmoid', 'first_units': 6, 'num_layers': 5, 'units_0': 7, 'units_1': 5, 'units_2': 7, 'units_3': 7, 'units_4': 9, 'units_5': 7, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0042'}


In [24]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5774 - accuracy: 0.7269 - 1s/epoch - 4ms/step
Loss: 0.5773822069168091, Accuracy: 0.7268804907798767
268/268 - 1s - loss: 0.5653 - accuracy: 0.7268 - 981ms/epoch - 4ms/step
Loss: 0.5653116106987, Accuracy: 0.7267638444900513
268/268 - 1s - loss: 0.5835 - accuracy: 0.7265 - 1s/epoch - 4ms/step
Loss: 0.5834793448448181, Accuracy: 0.7265306115150452


In [25]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 6,
 'num_layers': 5,
 'units_0': 1,
 'units_1': 1,
 'units_2': 9,
 'units_3': 9,
 'units_4': 9,
 'units_5': 3,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [26]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5774 - accuracy: 0.7269 - 675ms/epoch - 3ms/step
Loss: 0.5773822069168091, Accuracy: 0.7268804907798767


In [27]:
# Get second best model hyperparameters
second_hyper = tuner.get_best_hyperparameters(2)[1]
second_hyper.values

{'activation': 'tanh',
 'first_units': 6,
 'num_layers': 5,
 'units_0': 1,
 'units_1': 1,
 'units_2': 9,
 'units_3': 9,
 'units_4': 9,
 'units_5': 3,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0022'}

In [28]:
# Compare the performance to the second-best model
second_model = tuner.get_best_models(2)[1]
model_loss, model_accuracy = second_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5653 - accuracy: 0.7268 - 691ms/epoch - 3ms/step
Loss: 0.5653116106987, Accuracy: 0.7267638444900513


In [31]:
# Compare the performance to the second-best model
third_model = tuner.get_best_models(3)[2]
model_loss, model_accuracy = third_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5653 - accuracy: 0.7268 - 824ms/epoch - 3ms/step
Loss: 0.5653116106987, Accuracy: 0.7267638444900513


In [34]:
# Save the entire model as a `.keras`  or zip archive.
best_model.save('AlphabetSoupCharity_optimization_1_best.keras')
files.download('AlphabetSoupCharity_optimization_1_best.keras')

second_model.save('AlphabetSoupCharity_optimization_2.keras')
files.download('AlphabetSoupCharity_optimization_2.keras')

third_model.save('AlphabetSoupCharity_optimization_3.keras')
files.download('AlphabetSoupCharity_optimization_3.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

nn_model_2. Create aother model with an increased number of neurons in the irst hidden layer from 80 to 129 and the second hidden layer from 30 to 50.   

In [20]:
# Define the nn_model_2 with increased number of neurons.  129 in the first hidden layer, and 50 in the second hidden layer. 5
input_features = len(X_train[0])

hidden_nodes_layer1 =  129
hidden_nodes_layer2 = 50

nn_model_2 = tf.keras.models.Sequential()

# First hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# Second hidden layer
nn_model_2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_model_2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 129)               5676      
                                                                 
 dense_1 (Dense)             (None, 50)                6500      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 12227 (47.76 KB)
Trainable params: 12227 (47.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
# Compile the model
nn_model_2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model_2.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model_2 Loss: {model_loss}, Model_2 Accuracy: {model_accuracy}")


Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5339 - accuracy: 0.7416
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5344 - accuracy: 0.7418
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5340 - accuracy: 0.7412
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5332 - accuracy: 0.7421
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5327 - accuracy: 0.7425
Epoch 6/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5333 - accuracy: 0.7424
Epoch 7/100
804/804 [==============================] - 6s 7ms/step - loss: 0.5333 - accuracy: 0.7419
Epoch 8/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5329 - accuracy: 0.7420
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5329 - accuracy: 0.7422
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5332 - accura

In [35]:
# Save the entire model as a `.keras`  or zip archive.
nn_model_2.save('AlphabetSoupCharity_model_2.keras')
files.download('AlphabetSoupCharity_model_2.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

nn_model_3.  Create a Model_3, with fewer features

In [21]:
#  Import, read the charity_data.csv and save to a DataFrame 'application_df'.
import pandas as pd
application_2_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_2_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [23]:
# Drop the non-beneficial ID columns: EIN,'NAME, APPLICATION_TYPE, CLASSIFICATION, STATUS, SPECIAL_CONSIDERATIONS.
application_2_df.drop(application_2_df.columns[[0, 1, 2, 4, 7, 9]], axis=1, inplace=True)
application_2_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,ProductDev,Association,0,5000,1
1,Independent,Preservation,Co-operative,1-9999,108590,1
2,CompanySponsored,ProductDev,Association,0,5000,0
3,CompanySponsored,Preservation,Trust,10000-24999,6692,1
4,Independent,Heathcare,Trust,100000-499999,142590,1


In [24]:
application_2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   AFFILIATION    34299 non-null  object
 1   USE_CASE       34299 non-null  object
 2   ORGANIZATION   34299 non-null  object
 3   INCOME_AMT     34299 non-null  object
 4   ASK_AMT        34299 non-null  int64 
 5   IS_SUCCESSFUL  34299 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.6+ MB


In [25]:
## Generate our categorical variable lists that are with the dtype Object.
app_2_cat = application_2_df.dtypes[application_2_df.dtypes == "object"].index.tolist()
app_2_cat

['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
application_2_df = pd.get_dummies(application_2_df, columns=application_2_df.dtypes[application_2_df.dtypes == "object"].index.tolist())

# Display the modified DataFrame.
display(application_2_df.head())

,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,1,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0


In [28]:
# Split our preprocessed data into our features and target arrays.  X is the features. y is the target.
X = application_2_df.drop(columns = 'IS_SUCCESSFUL').values
y = application_2_df.IS_SUCCESSFUL.values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [29]:
# Display the shapes of the resulting datasets.
# there are 43 columns / Features
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (25724, 25)
X_test shape: (8575, 25)
y_train shape: (25724,)
y_test shape: (8575,)


In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [31]:
#model with fewer features  nn_model_3_3

input_features = len(X_train[0])

hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_model_3 = tf.keras.models.Sequential()

# First hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation="relu"))

# Second hidden layer
nn_model_3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_model_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                2080      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4541 (17.74 KB)
Trainable params: 4541 (17.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
# Compile the model
nn_model_3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Train the model
fit_model_3 = nn_model_3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 5ms/step - loss: 0.6094 - accuracy: 0.6933
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.6021 - accuracy: 0.6973
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6006 - accuracy: 0.6990
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.6005 - accuracy: 0.6990
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5996 - accuracy: 0.7004
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5993 - accuracy: 0.6995
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5989 - accuracy: 0.6988
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5985 - accuracy: 0.7006
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5988 - accuracy: 0.6999
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5983 - accura

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Model 3 Loss: {model_loss}, Model 3 Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6050 - accuracy: 0.6943 - 723ms/epoch - 3ms/step
Model 3 Loss: 0.605013370513916, Model 3 Accuracy: 0.6943440437316895


In [37]:
# Save the entire model as a `.keras`  or zip archive.
nn_model_3.save('AlphabetSoupCharity_model_3.keras')
files.download('AlphabetSoupCharity_model_3.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>